<a href="https://colab.research.google.com/github/aloml2543/KB_OCR/blob/main/src/model1_%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
https://github.com/qjadud1994/CRNN-Keras
"""

'\nhttps://github.com/qjadud1994/CRNN-Keras\n'

In [ ]:
import tensorflow as tf
import matplotlib
import numpy as np
import os
import zipfile
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import pickle
import datetime
import itertools
 
from tensorflow.keras import backend as K
 
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Input,GlobalMaxPooling2D,Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D, Reshape, LSTM, Lambda, add, concatenate
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
 
import numpy as np
import random
import multiprocessing
from cv2 import resize
from os import path, listdir
from google.colab.patches import cv2_imshow
import cv2
plt = matplotlib.pyplot

 
!python --version
print("tensorflow",tf.__version__)
print("numpy",np.__version__)
print("matplotlib",matplotlib.__version__)

Python 3.7.11
tensorflow 2.6.0
numpy 1.19.5
matplotlib 3.2.2


In [ ]:
def split_data(imgs, labels, split = 0.9, get = 1):
  get_point = int(len(imgs) * get)
  img_get = imgs[:get_point]
  lable_get = labels[:get_point]
  split_point = int(len(img_get) * split)

  return img_get[:split_point], lable_get[:split_point], img_get[split_point:], lable_get[split_point:]
#데이터셋 압축 해제
!unzip -n "/content/drive/MyDrive/KB_OCR/DATA/snukb_dataset.zip"  -d "/content/"

#dataset 가져오기
labels = pd.read_csv('/content/train/train.csv')
image_dir = [str(index) + '.jpg' for index in list(labels.index.to_numpy())]
labels = list(labels.label.to_numpy())

train_img, train_label, val_img, val_label = split_data(image_dir, labels, get = 0.1)


#파라미터들
model_path = '/content/drive/MyDrive/KB_OCR/model1-1/'
img_width = 256
img_height = 64
 
max_length = max([len(label) for label in labels])
characters = sorted(list(set(char for label in labels for char in label)))
num_classes = len(characters) + 1

print("최대 길이", max_length)
print("문자수", len(characters))
print("앞 20글자", characters[0:20])
print("뒤 20글자", characters[-20:])

print("\ntrain_img 수:", len(train_img))
print("train_label 수:", len(train_label))
print("val_img 수:", len(val_img))
print("val_label 수:", len(val_label))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/train/images/5507.jpg  
  inflating: /content/train/images/5508.jpg  
  inflating: /content/train/images/5509.jpg  
  inflating: /content/train/images/551.jpg  
  inflating: /content/train/images/5510.jpg  
  inflating: /content/train/images/5511.jpg  
  inflating: /content/train/images/5512.jpg  
  inflating: /content/train/images/5513.jpg  
  inflating: /content/train/images/5514.jpg  
  inflating: /content/train/images/5515.jpg  
  inflating: /content/train/images/5516.jpg  
  inflating: /content/train/images/5517.jpg  
  inflating: /content/train/images/5518.jpg  
  inflating: /content/train/images/5519.jpg  
  inflating: /content/train/images/552.jpg  
  inflating: /content/train/images/5520.jpg  
  inflating: /content/train/images/5521.jpg  
  inflating: /content/train/images/5522.jpg  
  inflating: /content/train/images/5523.jpg  
  inflating: /content/train/images/5524.jpg  
  inflating: /content/train/images/5525.jpg  


In [ ]:

def labels_to_text(labels):     # index형 데이터 -> 글자 변환
    return ''.join(list(map(lambda x: characters[int(x)], labels)))
 
def text_to_labels(text):      # 글자 -> index형 데이터 변환
    return list(map(lambda x: characters.index(x), text))

#입력값 제너레이터
class TextImageGenerator:
    def __init__(self, img_dirpath,img_dir,labels, img_w, img_h,
                 batch_size, downsample_factor, max_text_len=100):
        self.img_h = img_h
        self.img_w = img_w
        self.labels = labels
        self.batch_size = batch_size
        self.max_text_len = max_text_len
        self.downsample_factor = downsample_factor
        self.img_dirpath = img_dirpath                  # image dir path
        self.img_dir = img_dir     # images list
        self.n = len(self.img_dir)                      # number of images
        self.indexes = list(range(self.n))
        self.cur_index = 0
        self.imgs = np.zeros((self.n, self.img_h, self.img_w))
        self.texts = []
 
    ## samples의 이미지 목록들을 opencv로 읽어 저장하기, texts에는 label 저장
    def build_data(self):
        for i, img_file in enumerate(self.img_dir):
            img = cv2.imread(self.img_dirpath + img_file, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (self.img_w, self.img_h))
            img = img.astype(np.float32)
            img = (img / 255.0)
 
            self.imgs[i, :, :] = img
            self.texts.append(labels[i])
 
    def next_sample(self):      ## index max -> 0 으로 만들기
        self.cur_index += 1
        if self.cur_index >= self.n:
            self.cur_index = 0
            random.shuffle(self.indexes)
        return self.imgs[self.indexes[self.cur_index]], self.texts[self.indexes[self.cur_index]]
 
    def next_batch(self):       ## batch size만큼 가져오기
        while True:
            X_data = np.ones([self.batch_size, self.img_w, self.img_h, 1])     # (bs, 128, 64, 1)
            Y_data = np.zeros([self.batch_size, self.max_text_len])   # (bs, 9)
            input_length = np.ones((self.batch_size, 1)) * (self.img_w // self.downsample_factor - 2)  # (bs, 1)
            label_length = np.zeros((self.batch_size, 1))           # (bs, 1)
 
            for i in range(self.batch_size):
                img, text = self.next_sample()
                img = img.T
                img = np.expand_dims(img, -1)
                X_data[i] = img
                word = text_to_labels(text)
                for point in range(len(word)):
                    Y_data[i][point] = word[point]
                label_length[i] = len(text)
 
            # dict 형태로 복사
            inputs = {
                'the_input': X_data,  # (bs, 128, 64, 1)
                'the_labels': Y_data,  # (bs, 8)
                'input_length': input_length,  # (bs, 1) -> 모든 원소 value = 30
                'label_length': label_length  # (bs, 1) -> 모든 원소 value = 8
            }
            outputs = {'ctc': np.zeros([self.batch_size])}   # (bs, 1) -> 모든 원소 0
            yield (inputs, outputs)

train_file_path = '/content/train/images/'
train_dataset_generator = TextImageGenerator(train_file_path,tqdm(train_img), train_label, img_width, img_height, 1, 4, max_text_len=max_length)
train_dataset_generator.build_data()
val_dataset_generator = TextImageGenerator(train_file_path,tqdm(val_img), val_label, img_width, img_height, 1, 4, max_text_len=max_length)
val_dataset_generator.build_data()

100%|██████████| 301/301 [00:00<00:00, 2042.47it/s]


In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    y_pred = y_pred[:,2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
def get_Model(training=True):
    inputs = Input(name='the_input', shape=(img_width, img_height, 1), dtype='float32')
 
    # Convolution layer (VGG)
    inner = Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(inputs)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)
 
    inner = Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(inner)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)
    inner = Conv2D(256, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(inner)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(inner)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(1, 2), name='max3')(inner)
 
    inner = Conv2D(512, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(inner)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = Conv2D(512, (3, 3), padding='same', name='conv6')(inner)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(1, 2), name='max4')(inner)
 
    inner = Conv2D(512, (2, 2), padding='same', kernel_initializer='he_normal', name='con7')(inner)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
 
    # CNN to RNN
    inner = Reshape(target_shape=((64, 2048)), name='reshape')(inner)
    inner = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)
 
    # RNN layer
    lstm_1 = LSTM(256, return_sequences=True, kernel_initializer='he_normal', name='lstm1')(inner)
    lstm_1b = LSTM(256, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='lstm1_b')(inner)
    reversed_lstm_1b = Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (lstm_1b)
 
    lstm1_merged = add([lstm_1, reversed_lstm_1b])
    lstm1_merged = BatchNormalization()(lstm1_merged)
    
    lstm_2 = LSTM(256, return_sequences=True, kernel_initializer='he_normal', name='lstm2')(lstm1_merged)
    lstm_2b = LSTM(256, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='lstm2_b')(lstm1_merged)
    reversed_lstm_2b= Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (lstm_2b)
 
    lstm2_merged = concatenate([lstm_2, reversed_lstm_2b])
    lstm2_merged = BatchNormalization()(lstm2_merged)
 

    inner = Dense(num_classes, kernel_initializer='he_normal',name='dense2')(lstm2_merged)
    y_pred = Activation('softmax', name='softmax')(inner)
 
    labels = Input(name='the_labels', shape=[max_length], dtype='float32') 
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
 
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
 
    if training:
        return Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)
    else:
        return Model(inputs=[inputs], outputs=y_pred)
 
# Get the model
 
model = get_Model()
model.summary()
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=Adam(lr=0.02, beta_1=0.9, beta_2=0.999))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          [(None, 256, 64, 1)] 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 64, 64)  640         the_input[0][0]                  
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 64, 64)  256         conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 64, 64)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
checkpoint = ModelCheckpoint(filepath=model_path + '{epoch:02d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, mode='auto', save_best_only=True)
 
log_dir = model_path + "tensorbloard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
 
now = datetime.datetime.now()
print("시작 시간:", now)
history = model.fit(train_dataset_generator.next_batch(), steps_per_epoch=train_dataset_generator.n
                    ,validation_data=val_dataset_generator.next_batch(), validation_steps=val_dataset_generator.n
                    ,callbacks=[checkpoint, tensorboard_callback]
                    ,epochs=500)
now = datetime.datetime.now()
print("종료 시간:", now)

시작 시간: 2021-09-03 16:18:45.432812


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/500
2705/2705 [==============================] - 120s 36ms/step - loss: 34.3564 - val_loss: 47.8104

Epoch 00001: val_loss improved from inf to 47.81044, saving model to /content/drive/MyDrive/KB_OCR/model1-1/01-47.8104.hdf5
Epoch 2/500
2705/2705 [==============================] - 97s 36ms/step - loss: 45.0258 - val_loss: 46.2876

Epoch 00002: val_loss improved from 47.81044 to 46.28759, saving model to /content/drive/MyDrive/KB_OCR/model1-1/02-46.2876.hdf5
Epoch 3/500
2705/2705 [==============================] - 97s 36ms/step - loss: 45.0794 - val_loss: 979.3162

Epoch 00003: val_loss did not improve from 46.28759
Epoch 4/500
2705/2705 [==============================] - 96s 35ms/step - loss: 35.2462 - val_loss: 34.8826

Epoch 00004: val_loss improved from 46.28759 to 34.88258, saving model to /content/drive/MyDrive/KB_OCR/model1-1/04-34.8826.hdf5
Epoch 5/500
2705/2705 [==============================] - 96s 36ms/step - loss: 34.7607 - val_loss: 35.2198

Epoch 00005: val_loss di

KeyboardInterrupt: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

In [ ]:
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
predict_model = get_Model(training=False)
try:
    predict_model.load_weights('/content/drive/MyDrive/KB_OCR/model/model1-2.hdf5')
    print("...Previous weight data...")
except:
    raise Exception("No weight file!")
 
test_dir ='/content/test/images/'
test_imgs = os.listdir(test_dir)
now = datetime.datetime.now()
print("시작 시간:", now)

def decode_label(out):
    z, x, y = out.shape
    out = np.reshape(out,(int(x*y/num_classes),num_classes))
    word = []
    for letter in out:
      word.append(letter.argmax())
    out_best = [k for k, g in itertools.groupby(word)]
    outstr = ''
    for i in out_best:
        if i < len(characters):
            outstr += characters[i]
    return outstr

for test_img in test_imgs[:10]:
    img = cv2.imread(test_dir + test_img, cv2.IMREAD_GRAYSCALE)
    cv2_imshow(img)
 
    img_pred = cv2.resize(img, (img_width, img_height))
    img_pred = img_pred.astype(np.float32)
    img_pred = (img_pred / 255.0)
    img_pred = img_pred.T
    img_pred = np.expand_dims(img_pred, axis=-1)
    img_pred = np.expand_dims(img_pred, axis=0)
 
    net_out_value = predict_model.predict(img_pred)
    pred_texts = decode_label(net_out_value)
    print('Predicted: %s ' % (pred_texts))
    